<a href="https://colab.research.google.com/github/neuralHIVEmind/skills-introduction-to-github/blob/main/Oculus_Reparo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# 🧙‍♂️ AI MAGIC STUDIO - PRO EDITION (FIXED!) 🧙‍♂️
print("🔥 Loading PRO-Level AI Magic Studio...")

!pip install -q transformers accelerate torchvision pillow gradio opencv-python diffusers
import gradio as gr
import numpy as np
from PIL import Image
import cv2
import torch

# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 Using: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# Load REAL Hugging Face models - USING A DIFFERENT MODEL THAT DOESN'T NEED PROMPTS
print("⏳ Loading Swin2SR Model from Hugging Face...")
from transformers import pipeline
upscaler = pipeline("image-to-image",
                   model="caidas/swin2SR-realworld-sr-x4-64-bsrgan-psnr",  # BETTER MODEL, NO PROMPT NEEDED
                   device=0 if torch.cuda.is_available() else -1,
                   torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
print("✅ Swin2SR Model Loaded!")

print("⏳ Loading AI Detector from Hugging Face...")
detector = pipeline("image-classification",
                   model="umm-maybe/AI-image-detector",  # REAL HUGGING FACE MODEL
                   device=0 if torch.cuda.is_available() else -1)
print("✅ AI Detector Loaded!")

def enhance_image(input_image):
    """Enhance image with REAL AI upscaling"""
    if input_image is None:
        return None, None, "Upload image first!", "0%", "0%"

    try:
        # Convert to appropriate format
        input_image = input_image.convert("RGB")

        # REAL AI UPSCALING - NO PROMPT NEEDED!
        print("🎨 Enhancing image with AI...")
        enhanced = upscaler(input_image)  # SIMPLE CALL, NO PROMPT!

        # REAL AI DETECTION
        print("🔍 Analyzing image with AI...")
        detection = detector(enhanced)
        human_conf = next((d['score'] for d in detection if 'human' in d['label'].lower()), 0)
        ai_conf = next((d['score'] for d in detection if 'artificial' in d['label'].lower()), 0)

        human_meter = f"👤 Human: {human_conf*100:.1f}%"
        ai_meter = f"🤖 AI: {ai_conf*100:.1f}%"
        result_text = f"🎯 Confidence: Human {human_conf*100:.1f}% | AI {ai_conf*100:.1f}%"

        return input_image, enhanced, result_text, human_meter, ai_meter

    except Exception as e:
        return None, None, f"❌ Error: {str(e)}", "0%", "0%"

def create_comparison_image(original, enhanced, slider_value):
    """Create magic curtain effect between original and AI-enhanced"""
    if original is None or enhanced is None:
        return None

    # Convert images to numpy arrays
    orig_np = np.array(original)
    enh_np = np.array(enhanced)

    # Resize both to same dimensions for comparison
    height, width = orig_np.shape[:2]
    enh_np = cv2.resize(enh_np, (width, height))  # Only for visual comparison!

    # Create split image based on slider
    split_point = int(width * slider_value / 100)

    # Create comparison image (curtain effect)
    comparison = orig_np.copy()
    comparison[:, split_point:] = enh_np[:, split_point:]

    # Add division line
    cv2.line(comparison, (split_point, 0), (split_point, height), (255, 255, 255), 3)
    cv2.line(comparison, (split_point, 0), (split_point, height), (0, 0, 0), 1)

    # Add text labels
    cv2.putText(comparison, "Original", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(comparison, "AI Enhanced", (width - 120, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    return Image.fromarray(comparison)

def process_with_slider(input_image, slider_value):
    """Process image with curtain effect"""
    if input_image is None:
        return None, "Upload image first!", "0%", "0%"

    original, enhanced, result_text, human_meter, ai_meter = enhance_image(input_image)

    if enhanced is None:
        return None, result_text, human_meter, ai_meter

    # Create comparison image with curtain effect
    comparison = create_comparison_image(original, enhanced, slider_value)

    return comparison, result_text, human_meter, ai_meter

# Create the interface
print("🎨 Creating Pro Interface...")

with gr.Blocks(title="✨ AI Magic Studio Pro", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🧙‍♂️ AI Magic Studio Pro
    *Real AI Upscaling with Hugging Face Models!*
    """)

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 🎯 Upload & Control")
            image_input = gr.Image(label="📷 Upload Image", type="pil", height=300)
            process_btn = gr.Button("✨ AI Enhance", variant="primary", size="lg")

            gr.Markdown("### 🪄 Magic Curtain")
            curtain_slider = gr.Slider(
                minimum=0,
                maximum=100,
                value=50,
                step=1,
                label="Drag to compare",
                info="← Original | AI Enhanced →"
            )

        with gr.Column(scale=2):
            gr.Markdown("### 🔮 AI Enhanced Result")
            result_image = gr.Image(
                label="Original vs AI Enhanced",
                interactive=False,
                height=400,
                show_download_button=True
            )

            with gr.Row():
                with gr.Column():
                    gr.Markdown("### 🔍 AI Detection")
                    result_text = gr.Textbox(label="Analysis Result", interactive=False)
                    human_output = gr.Textbox(label="Human Confidence", interactive=False)
                    ai_output = gr.Textbox(label="AI Confidence", interactive=False)

    # Connect everything
    process_btn.click(
        fn=enhance_image,
        inputs=[image_input],
        outputs=[image_input, result_image, result_text, human_output, ai_output]
    )

    curtain_slider.change(
        fn=process_with_slider,
        inputs=[image_input, curtain_slider],
        outputs=[result_image, result_text, human_output, ai_output]
    )

# Launch with port handling
print("🎉 Launching AI Magic Studio Pro...")

# Try multiple ports
for port in [7860, 9000, 8000, 7000]:
    try:
        demo.launch(share=True, server_name="0.0.0.0", server_port=port, debug=False)
        print(f"✅ Successfully launched on port {port}")
        break
    except OSError:
        print(f"Port {port} busy, trying next...")
        continue
else:
    print("Using automatic port selection...")
    demo.launch(share=True, server_name="0.0.0.0", server_port=0, debug=False)

🔥 Loading PRO-Level AI Magic Studio...
🚀 Using: Tesla T4
⏳ Loading Swin2SR Model from Hugging Face...


config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/48.5M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

Device set to use cuda:0


...

Device set to use cuda:0


✅ AI Detector Loaded!
🎨 Creating Pro Interface...
🎉 Launching AI Magic Studio Pro...
Port 7860 busy, trying next...


ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 9000): [errno 98] address already in use


Port 9000 busy, trying next...
Port 8000 busy, trying next...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ef2cbaea91e8ebe9e8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✅ Successfully launched on port 7000
